In [169]:
from enum import unique
import json
import re
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, GRU
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
import os

In [170]:
with open("../input/russian-poems2/classic_poems.json", "r", encoding='utf8') as read_file:
    data = json.load(read_file)

poems = []
for obj in data:
    poems.append(obj['content'])

text = ''.join(poems)



In [171]:
chars = sorted(set(text))

chars2idx = {c:i for i, c in enumerate(chars)}
idx2char = np.array(chars)

input_len = len(text)
vocab_len = len(chars)
print(f"{input_len} inp, {vocab_len} vocab")

# I started to use tf dataset to simplify the process
# First it got a all text as list of numbers
input_dataset = tf.data.Dataset.from_tensor_slices([chars2idx[i] for i in text])
seq_length = 300

# Second, I made batches of seq_length 
sequences = input_dataset.batch(seq_length+1, drop_remainder = True)

# now it holds tuples ((100), (100)) where first is x, second is y
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text
dataset = sequences.map(split_input_target)


1968220 inp, 176 vocab


In [172]:
# shuffle and split to batches again
batch_size = 32
embedding_dim = 200
rnn_units = 1000

dataset = dataset.shuffle(1000).batch(batch_size, drop_remainder=True)

In [173]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True),
    tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [174]:
model = build_model(vocab_len, embedding_dim, rnn_units, batch_size)
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer='adam', loss=loss)

In [175]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

model.fit(dataset, epochs=50, callbacks=[checkpoint_callback])
model.save('model1')

Epoch 1/50
204/204 [==============================] - 22s 107ms/step - loss: 3.0143
Epoch 2/50
204/204 [==============================] - 22s 108ms/step - loss: 2.4922
Epoch 3/50
204/204 [==============================] - 22s 108ms/step - loss: 2.3294
Epoch 4/50
204/204 [==============================] - 22s 108ms/step - loss: 2.1981
Epoch 5/50
204/204 [==============================] - 22s 107ms/step - loss: 2.0866
Epoch 6/50
204/204 [==============================] - 22s 108ms/step - loss: 2.0010
Epoch 7/50
204/204 [==============================] - 22s 108ms/step - loss: 1.9336
Epoch 8/50
204/204 [==============================] - 22s 109ms/step - loss: 1.8788
Epoch 9/50
204/204 [==============================] - 22s 108ms/step - loss: 1.8345
Epoch 10/50
204/204 [==============================] - 22s 108ms/step - loss: 1.7953
Epoch 11/50
204/204 [==============================] - 22s 108ms/step - loss: 1.7623
Epoch 12/50
204/204 [==============================] - 22s 107ms/step - lo

In [176]:
model = build_model(vocab_len, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.save('/model1')

In [181]:

def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
    num_generate = 300

  # Converting our start string to numbers (vectorizing)
    input_eval = [chars2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    print(input_eval.shape)
  # Empty string to store our results
    text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
    temperature = 0.1

  # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
    # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [185]:

print(generate_text(model, start_string=u"Дивный "))

(1, 7)
Дивный из нас,
Уже не вырваться с ней?
Когда под забором в дом
Там – в сердце – небесный свет,
Но старинные объятия,
Но в отдых открытых видений
Встречал ее в очи глянулся,
И оба покрыл от скуки
Открыли дня своей дороги
Не в силах страсти и в стихах.
Жизнь в сердце – не находя чужи,
Над ним собаки не надо.
